https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python <br>
http://www.aclweb.org/anthology/E09-3003 <br>
http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.323.6525&rep=rep1&type=pdf <br>
https://www.udemy.com/natural-language-processing-with-deep-learning-in-python/learn/v4/questions/4267184

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[]


In [2]:
import nltk
from nltk.corpus import brown
from collections import Counter

In [4]:
news_text = brown.sents(categories='news')

In [5]:
news_text_lowered=[[each_word.lower() for each_word in each_sent] for each_sent in news_text]

In [6]:
news_text_lowered_list=[each_word for each_sent in news_text_lowered for each_word in each_sent]
Unique_words=list(set(news_text_lowered_list))
V=len(Unique_words)

In [24]:
#creating X (Input - last word) and Y (Target - current word) vectors

def get_sentences_with_word2idx():
    sentences = news_text_lowered[0:20]
    indexed_sentences = []

    i = 2
    word2idx = {'START': 0, 'END': 1}
    for sentence in sentences:
        indexed_sentence = []
        for token in sentence:
            token = token.lower()
            if token not in word2idx:
                word2idx[token] = i
                i += 1
            indexed_sentence.append(word2idx[token])
        indexed_sentences.append(indexed_sentence)

    print("Vocab size:", i)
    return indexed_sentences, word2idx

sentences, word2idx = get_sentences_with_word2idx()

Vocab size: 238


In [25]:
start_idx = word2idx['START']
end_idx = word2idx['END']

# train a logistic model
W = np.random.randn(V, V) / np.sqrt(V)

lr = 1e-1
losses = []
def softmax(a):
    a = a - a.max()
    exp_a = np.exp(a)
    return exp_a / exp_a.sum(axis=1, keepdims=True)

In [26]:
print (len(sentences))

20


In [27]:
start_idx = word2idx['START']
end_idx = word2idx['END']
each=0
for sentence in sentences:
    each+=1
    print (each)
    # convert sentence into one-hot encoded inputs and targets
    sentence = [start_idx] + sentence + [end_idx]
    n = len(sentence)
    inputs = np.zeros((n - 1, V))
    targets = np.zeros((n - 1, V))
    inputs[np.arange(n - 1), sentence[:n-1]] = 1
    targets[np.arange(n - 1), sentence[1:]] = 1
    # get output predictions
    predictions = softmax(inputs.dot(W))
    # do a gradient descent step
    W = W - lr * inputs.T.dot(predictions - targets)
    # keep track of the loss
    loss = -np.sum(targets * np.log(predictions)) / (n - 1)
    losses.append(loss)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [29]:
def logistic_bigram(new_sent):
    new_sent=new_sent.lower()
    new_words=new_sent.split()
    new_words=[start_idx] + new_words + [end_idx]
    new_words_index=[]
    for each in new_words:
        if each in word2idx.keys():
            new_words_index.append(word2idx[each])
        else:
            new_words_index.append(999)
    print (new_words_index)
    n = len(new_words_index)
    inputs_new = np.zeros((n - 1, V))
    inputs_new[np.arange(n - 1), new_words_index[:n-1]] = 1
    predictions = softmax(inputs_new.dot(W))
    return predictions.sum()

In [16]:
print (news_text_lowered[0])

['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', "atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']


In [32]:
print (logistic_bigram("the fulton county jury said"))
print (logistic_bigram("some randomui statementio is thisp"))
print (logistic_bigram('only a relative handful of such reports'))
print (logistic_bigram("ramdndomm fhlkh kjkel kjlk"))

[999, 2, 3, 4, 6, 7, 999]
6.0
[999, 215, 999, 999, 138, 999, 999]
6.0
[999, 68, 69, 70, 71, 11, 72, 60, 999]
8.0
[999, 999, 999, 999, 999, 999]
5.0
